In [2]:
%matplotlib inline 
#有了matplotlib inline 就可以省掉plt.show()了
%load_ext autoreload
%autoreload 2

from __future__ import division
from imports import *
#import pylab #使用pylab模式，可以直接使用numpy、pandas和绘图，而不需要再次引入,会配置为使用你所指定的matplotlib gui后端，还会提供一种更接近于matlab的界面
#%pylab
from IPython.display import display_html
display_html("""<button onclick="$('.input, .prompt, .output_stderr, .output_error').toggle();">Toggle code</button>""", raw=True)
# 允许多行输出
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'
from sklearn import *
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Toggle code

## 集成算法
将多种机器学习算法组合在一起，提高算法准确度

### 集成的方法
* 装袋算法（Bagging）：先将训练集分离成多个子集，然后通过各个子集训练多个模型
* 提升算法（Boosting）：训练多个模型并组成一个序列，序列中的每个模型都会修正前一个模型的错误
* 投票算法（Voting）：训练多个模型，并采用样本统计来提高模型的准确度

### 装袋算法
通过给定组合投票的方式获得最优解

#### 装袋决策树

In [9]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
filename = './csvs/user_label_7day_0301.csv'
datasource = pd.read_csv(filename)
data = datasource[datasource['user_label_7day.fplatformname']=='新浪平台']
array = data.values
X = array[:,5:9].astype(np.int64)
Y = array[:,9].astype(np.int64)
num_folds = 10
seed = 7
kfold = KFold(n_splits=num_folds,random_state=seed)

In [11]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

cart = DecisionTreeClassifier()
num_tree = 100
model = BaggingClassifier(base_estimator=cart,n_estimators=num_tree,random_state=seed)
result = cross_val_score(model,X,Y,cv=kfold)
print result

[ 0.95625    0.9625     0.9375     0.95625    0.925      0.96875    0.95
  0.93125    0.93125    0.9245283]


#### 随机森林
用随机的方式建立很多决策树，每一颗决策树之间是没有关联的。  
当有一个新的输入样本进入的时候，就让每一颗树分别进行判断，看这个样本应该属于哪一类，然后被选得最多的那一类，就是这个样本的分类

In [13]:
from sklearn.ensemble import RandomForestClassifier
num_tree = 100
max_features = 3
model = RandomForestClassifier(n_estimators=num_tree,random_state=seed,max_features=max_features)
result = cross_val_score(model,X,Y,cv=kfold)
print result.mean()

0.945577830189


#### 极端随机树
和随机森林很像，都是由很多决策树构成  
区别在于：  
* 随机森林用的是Bagging模型，但是极端随机树应用的是相同的全部训练样本
* 极端随机树是完全随机地选择分叉特征属性的

In [15]:
from sklearn.ensemble import ExtraTreesClassifier
model = ExtraTreesClassifier(n_estimators=num_tree,random_state=seed,max_features=max_features)
result = cross_val_score(model,X,Y,cv=kfold)
result.mean()

0.93182389937106913

### 提升算法
是一种用来提高弱分类算法准确度的方法  
这种方法先构造一个预测函数系列，然后以一定的方式将它们组成一个预测函数  
主要通过对样本集的操作获得样本子集，然后用弱分类算法在样本子集上训练生成一系列的基分类器。  
它可以用来提高其他弱分类算法的识别率

#### AdaBoost
是一种迭代算法，核心思想是针对同一个训练集训练不同的分类器，然后把这些弱分类器集合起来，构成一个更强的最终分类器。通过修改权值

In [17]:
from sklearn.ensemble import AdaBoostClassifier
model = AdaBoostClassifier(n_estimators=num_tree,random_state=seed)
result = cross_val_score(model,X,Y,cv=kfold)
result.mean()

0.95245283018867932

#### 随机梯度提升（GBM）
梯度算子总是指向函数值增长最快的方向，一次只用一个样本点来更新回归系数

In [18]:
from sklearn.ensemble import GradientBoostingClassifier
model = GradientBoostingClassifier(n_estimators=num_tree,random_state=seed)
result = cross_val_score(model,X,Y,cv=kfold)
result.mean()

0.93744103773584908

## 投票算法（Voting）
通过创建两个或多个算法模型，利用投票算法将这些算法包装起来，计算各个子模型的平均预测状况

In [20]:
from sklearn.ensemble import VotingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
cart = DecisionTreeClassifier()
models = []
model_logistic = LogisticRegression()
models.append(('logistic',model_logistic))
model_cart = DecisionTreeClassifier()
models.append(('cart',model_cart))
model_svc = SVC()
models.append(('svm',model_svc))
ensemble_model = VotingClassifier(estimators=models)
result = cross_val_score(ensemble_model,X,Y,cv=kfold)
print result.mean()

0.952452830189
